In [9]:
import pandas as pd

# Loading the dataset
cleaned_df = pd.read_csv('../data/cleaned/cleaned_dataset.csv')

# Quick look at the structure
print(cleaned_df.shape)
print(cleaned_df.dtypes)
dashboard_df = cleaned_df.copy()
dashboard_df.head()

(54966, 15)
name                   object
age                     int64
gender                 object
blood_type             object
medical_condition      object
date_of_admission      object
doctor                 object
hospital               object
insurance_provider     object
billing_amount        float64
room_number             int64
admission_type         object
discharge_date         object
medication             object
test_results           object
dtype: object


,name,age,gender,blood_type,medical_condition,date_of_admission,doctor,hospital,insurance_provider,billing_amount,room_number,admission_type,discharge_date,medication,test_results
0,Bobby Jackson,30,Male,B-,Cancer,2024-01-31,Matthew Smith,Sons and Miller,Blue Cross,18856.28,328,Urgent,2024-02-02,Paracetamol,Normal
1,Leslie Terry,62,Male,A+,Obesity,2019-08-20,Samantha Davies,Kim Inc,Medicare,33643.33,265,Emergency,2019-08-26,Ibuprofen,Inconclusive
2,Danny Smith,76,Female,A-,Obesity,2022-09-22,Tiffany Mitchell,Cook PLC,Aetna,27955.10,205,Emergency,2022-10-07,Aspirin,Normal
3,Andrew Watts,28,Female,O+,Diabetes,2020-11-18,Kevin Wells,Hernandez Rogers and Vang,Medicare,37909.78,450,Elective,2020-12-18,Ibuprofen,Abnormal
4,Adrienne Bell,43,Female,AB+,Cancer,2022-09-19,Kathleen Hanna,White-White,Aetna,14238.32,458,Urgent,2022-10-09,Penicillin,Abnormal


---

In [2]:
### Feature #1: Patient ID

import hashlib

# Condition abbreviation map
condition_map = {
    'Cancer': 'CA',
    'Obesity': 'OB',
    'Diabetes': 'DB',
    'Asthma': 'AS',
    'Hypertension': 'HT',
    'Arthritis': 'AR'
}

def create_patient_id(row):

    # ============== Parameters ==============

    # First and Last Name
    full_name = row['name'].strip().split()
    first_initial = full_name[0][0].upper() if len(full_name) > 0 else 'X'
    last_initial = full_name[1][0].upper() if len(full_name) > 1 else 'X'

    # Room Number
    room = str(row['room_number']) if pd.notnull(row['room_number']) else '000'

    # Medical Condition
    condition_abbr = condition_map.get(row['medical_condition'], 'XX')  # Use a predefined map

    # ============== Adding the short hash ==============

    # Putting all parameters together to create the unique ID
    base_id = f"{first_initial}{last_initial}{room}{condition_abbr}"

    hash_suffix = hashlib.md5(base_id.encode()).hexdigest()[:3].upper()

    return f"{base_id}_{hash_suffix}"

dashboard_df['patient_id'] = dashboard_df.apply(create_patient_id, axis=1)
dashboard_df.head()

,name,age,gender,blood_type,medical_condition,date_of_admission,doctor,hospital,insurance_provider,billing_amount,room_number,admission_type,discharge_date,medication,test_results,patient_id
0,Bobby Jackson,30,Male,B-,Cancer,2024-01-31,Matthew Smith,Sons and Miller,Blue Cross,18856.28,328,Urgent,2024-02-02,Paracetamol,Normal,BJ328CA_628
1,Leslie Terry,62,Male,A+,Obesity,2019-08-20,Samantha Davies,Kim Inc,Medicare,33643.33,265,Emergency,2019-08-26,Ibuprofen,Inconclusive,LT265OB_C0E
2,Danny Smith,76,Female,A-,Obesity,2022-09-22,Tiffany Mitchell,Cook PLC,Aetna,27955.10,205,Emergency,2022-10-07,Aspirin,Normal,DS205OB_EF6
3,Andrew Watts,28,Female,O+,Diabetes,2020-11-18,Kevin Wells,Hernandez Rogers and Vang,Medicare,37909.78,450,Elective,2020-12-18,Ibuprofen,Abnormal,AW450DB_991
4,Adrienne Bell,43,Female,AB+,Cancer,2022-09-19,Kathleen Hanna,White-White,Aetna,14238.32,458,Urgent,2022-10-09,Penicillin,Abnormal,AB458CA_E7B


In [3]:
# Ensuring the unique ID is at the start of the dataset
cols = ['patient_id'] + [col for col in dashboard_df.columns if col != 'patient_id']
dashboard_df = dashboard_df[cols]
dashboard_df.head()

,patient_id,name,age,gender,blood_type,medical_condition,date_of_admission,doctor,hospital,insurance_provider,billing_amount,room_number,admission_type,discharge_date,medication,test_results
0,BJ328CA_628,Bobby Jackson,30,Male,B-,Cancer,2024-01-31,Matthew Smith,Sons and Miller,Blue Cross,18856.28,328,Urgent,2024-02-02,Paracetamol,Normal
1,LT265OB_C0E,Leslie Terry,62,Male,A+,Obesity,2019-08-20,Samantha Davies,Kim Inc,Medicare,33643.33,265,Emergency,2019-08-26,Ibuprofen,Inconclusive
2,DS205OB_EF6,Danny Smith,76,Female,A-,Obesity,2022-09-22,Tiffany Mitchell,Cook PLC,Aetna,27955.10,205,Emergency,2022-10-07,Aspirin,Normal
3,AW450DB_991,Andrew Watts,28,Female,O+,Diabetes,2020-11-18,Kevin Wells,Hernandez Rogers and Vang,Medicare,37909.78,450,Elective,2020-12-18,Ibuprofen,Abnormal
4,AB458CA_E7B,Adrienne Bell,43,Female,AB+,Cancer,2022-09-19,Kathleen Hanna,White-White,Aetna,14238.32,458,Urgent,2022-10-09,Penicillin,Abnormal


---

In [4]:
### Feature #2: Length of Stay

# Ensuring date columns are datetime
dashboard_df['date_of_admission'] = pd.to_datetime(dashboard_df['date_of_admission'])
dashboard_df['discharge_date'] = pd.to_datetime(dashboard_df['discharge_date'])

# Calculating length of stay and handling negative/zero values
dashboard_df['length_of_stay'] = (dashboard_df['discharge_date'] - dashboard_df['date_of_admission']).dt.days
dashboard_df['length_of_stay'] = dashboard_df['length_of_stay'].apply(lambda x: max(x, 1))
dashboard_df.head()

,patient_id,name,age,gender,blood_type,medical_condition,date_of_admission,doctor,hospital,insurance_provider,billing_amount,room_number,admission_type,discharge_date,medication,test_results,length_of_stay
0,BJ328CA_628,Bobby Jackson,30,Male,B-,Cancer,2024-01-31,Matthew Smith,Sons and Miller,Blue Cross,18856.28,328,Urgent,2024-02-02,Paracetamol,Normal,2
1,LT265OB_C0E,Leslie Terry,62,Male,A+,Obesity,2019-08-20,Samantha Davies,Kim Inc,Medicare,33643.33,265,Emergency,2019-08-26,Ibuprofen,Inconclusive,6
2,DS205OB_EF6,Danny Smith,76,Female,A-,Obesity,2022-09-22,Tiffany Mitchell,Cook PLC,Aetna,27955.10,205,Emergency,2022-10-07,Aspirin,Normal,15
3,AW450DB_991,Andrew Watts,28,Female,O+,Diabetes,2020-11-18,Kevin Wells,Hernandez Rogers and Vang,Medicare,37909.78,450,Elective,2020-12-18,Ibuprofen,Abnormal,30
4,AB458CA_E7B,Adrienne Bell,43,Female,AB+,Cancer,2022-09-19,Kathleen Hanna,White-White,Aetna,14238.32,458,Urgent,2022-10-09,Penicillin,Abnormal,20


---

In [5]:
### Feature #3: Projected Treatment Cost

# Defining cost mappings with an increased base cost to better reflect real-world pricing
base_cost_per_day = 800

condition_cost = {
    'Cancer': 3000, 'Obesity': 1500, 'Diabetes': 2100,
    'Asthma': 1200, 'Hypertension': 1800, 'Arthritis': 1050
}

medication_cost = {
    'Paracetamol': 80, 'Ibuprofen': 100, 'Aspirin': 90,
    'Penicillin': 140, 'Lipitor': 250
}

test_multiplier = {
    'Normal': 1.0, 'Inconclusive': 1.2, 'Abnormal': 1.6
}

# Treatment cost function to calculate final treatment cost based on different parameters
def calculate_treatment_cost(row):
    base = base_cost_per_day * row['length_of_stay']
    cond = condition_cost.get(row['medical_condition'], 400)
    med = medication_cost.get(row['medication'], 100)
    mult = test_multiplier.get(row['test_results'], 1.0)
    return round((base + cond + med) * mult, 2)

# Testing the reults to compare whether we're going the right direction with projected costs or not
dashboard_df['treatment_cost'] = dashboard_df.apply(calculate_treatment_cost, axis=1)
dashboard_df[['billing_amount', 'treatment_cost']]

,billing_amount,treatment_cost
0,18856.28,4680.0
1,33643.33,7680.0
2,27955.10,13590.0
3,37909.78,41920.0
4,14238.32,30624.0
...,...,...
54961,2650.71,40544.0
54962,31457.80,8790.0
54963,27620.76,38880.0
54964,32451.09,9520.0


In [6]:
dashboard_df.rename(columns={'treatment_cost': 'projected_treatment_cost'}, inplace=True)

# This helps us see the gaps between the real and projected values.
dashboard_df['cost_gap'] = (dashboard_df['billing_amount'] - dashboard_df['projected_treatment_cost']).round(2)

print(dashboard_df['cost_gap'].describe())
print(dashboard_df[['billing_amount', 'projected_treatment_cost', 'cost_gap']].head())

# Flagginf rows where the difference is too large for later review
dashboard_df['cost_gap_flag'] = dashboard_df['cost_gap'].apply(lambda x: 'High' if abs(x) > 10000 else 'OK')

dashboard_df['cost_gap_flag']

count    54966.000000
mean      7412.055196
std      17222.194547
min     -42808.490000
25%      -5176.555000
50%       7404.120000
75%      20407.220000
max      49380.040000
Name: cost_gap, dtype: float64
   billing_amount  projected_treatment_cost  cost_gap
0        18856.28                    4680.0  14176.28
1        33643.33                    7680.0  25963.33
2        27955.10                   13590.0  14365.10
3        37909.78                   41920.0  -4010.22
4        14238.32                   30624.0 -16385.68


0        High
1        High
2        High
3          OK
4        High
         ... 
54961    High
54962    High
54963    High
54964    High
54965    High
Name: cost_gap_flag, Length: 54966, dtype: object

---

In [7]:
### Feature #4: Readmission Risk Flag

# Sorting DataFrame by patient ID and admission date
df_sorted = dashboard_df.sort_values(by=['patient_id', 'date_of_admission']).reset_index(drop=True)

# Initializing the readmission flag
df_sorted['readmitted_within_30_days'] = False

# Iterating through each patient group
for patient_id, group in df_sorted.groupby('patient_id'):
    admission_dates = group['date_of_admission'].sort_values().reset_index()
    diffs = admission_dates['date_of_admission'].diff().dt.days  # Calculating days between admissions
    readmitted_mask = diffs <= 30  # Checking if the difference is 30 days or less
    readmitted_indices = admission_dates.loc[readmitted_mask.fillna(False), 'index']  # Getting indices of readmissions
    df_sorted.loc[readmitted_indices, 'readmitted_within_30_days'] = True  # Marking readmissions

# Calculating total readmissions per patient
readmission_counts = df_sorted.groupby('patient_id')['readmitted_within_30_days'].sum().reset_index()
readmission_counts.columns = ['patient_id', 'readmission_count']

# Merging readmission count into main DataFrame
dashboard_df = pd.merge(dashboard_df, readmission_counts, on='patient_id', how='left')

# Assigning simplified flag based on whether any readmission occurred
dashboard_df['readmission_risk_flag'] = dashboard_df['readmission_count'].apply(
    lambda x: 'Readmitted' if x > 0 else 'Not Readmitted'
)

In [8]:
# Saving final dataset
dashboard_df.to_csv("../powerbi/data/dashboard_dataset.csv", index=False)

print("✅ Feature Engineering complete! Processed data saved in 'powerbi/data/dashboard_dataset.csv'.")

✅ Feature Engineering complete! Processed data saved in 'powerbi/data/dashboard_dataset.csv'.
